In [1]:
# Loading all the necessary Python libraries. 
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr

In [2]:
"""
Input: List of numbers
Output: The average of members within the list.
Description: This function helps when I apply conditions to the dataframe, such as separating students into those
that attend public and private schools. In some instances, the length of the dataframe is 0, and dividing 0 gives
an error. Because I prefer to abstract from checking the type of elements within each array, I create this function. 
""" 

def avg(lst):
    count = 0
    sum_val = 0
    for member in lst:
        try:
            temp = float(member)
            count += 1
            sum_val += temp
        except:
            temp = 0
    try:
        return sum_val/count
    except:
        return 0

In [3]:
"""
Input: A list of two mixed data structure: a string of float and a float.
Output: A converted list where every string of float is converted into a float. Else, the list remains the same. 
"""

def create_float_lst(lst):
    temp_lst = []
    for member in lst:
        if type(member) == float:
            temp_lst.append(member)
        else:
            try: 
                temp_lst.append(float(member))
            except:
                temp_lst.append(member)
    return temp_lst

In [4]:
# Loading the dataframe from the .csv file on PISA 2018 data and dropping any unnecessary rows. 
# Note: This data is already cleaned in the previous ipnyb file. 

thadf = pd.read_csv("pisa2018_studentindex_perf.csv").dropna(how = "all", axis = 1).fillna(method = "ffill").fillna(method = "bfill")
thadf.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [5]:
subject_lst = ["math", "scie", "read"]

In [6]:
# Reading the Stata file which explains what the acronym for each column means. 
"""
Note: I drew most of the ideas from this website:
https://stackoverflow.com/questions/44809696/is-there-a-way-to-read-stata-labels-in-python
"""

reader = pd.io.stata.StataReader("PISA_2018_THA.dta")
header = reader.variable_labels()
for var in header:
        name  = var
        label = header[name]

In [7]:
# Right now, header is our dictionary where its keys are each column's title and its values are each column's 
# interpretation. Because we have dropped some columns, we will edit the header to ensure we do not accidentally
# analyze nonexistent columns. 

header_temp = {}
for key in list(thadf.columns.tolist()):
    try:
        header_temp[key] = header[key]
    except:
        None

header = header_temp 

In [8]:
# Creating a flipped dictionary where the key is the interpretation (e.g. fear of failure) and the value is 
# its corresponding title (e.g. gfofail). This dictionary is created for future convenience. 

header_flip = {}
for key in list(header.keys()):
    header_flip[header[key]] = key

In [9]:
# Creating the lists of WLE indices. Each index is grouped into a larger list according to major factors that may
# influence education outcome: home background, self-learning experience, parents, schools, attitudes, and so on.
# The name of each list explains how I categorize different factors. 
"""
Explanation: I have experimented with running regression with Y (dependent variable) as education outcome and X (predictor 
variable) as students' responses to questions on attitude towards education. The result is that those questions 
could not capture the difference in achievement. Moreover, factors besides attitude, such as socioeconomic background,
parents' level of education, and so on, are likely to affect the education outcome as well. Because there are simply
too many questions to convert into 1-4 scale or dummy variables, I chose to use the Warm's Likelihood Estimate (WLE)
weighted to mitigate the effect of regional difference on answers to the survey. 

Note: I do not include any WLE indices related to ICT because when I cleaned the data, I found out that ICT-related 
questions contained too many 'No Responses.' Therefore, I removed those indices so that the analysis would not 
have imbalanced set of data. Also, I do not include the class size because it is already captured by the student
to teacher ratio. 
""" 

home_index = ["homepos","cultposs","hedres","wealth","ictres"]
self_index = ["joyread","screadcomp","screaddiff","pisadiff"]
parent_index = ["emosups","cursupp","emosupp","pqschool","presupp","joyreadp","soconpa"]
school_index = ["disclima","teachsup","dirins","stimread","adaptivity","teachint","paschpol"]
attitude_index = ["perfeed","percomp","percoop","attlnact", "compete","workmast","gfofail","eudmo"]
attitude_index += ["swbp","resilience","mastgoal","bodyima"]
belonging_index = ["discrim","belong","beingbullied"]
career_index = ["infocar","infojob1","infojob2"]
socio_status = ["escs"]
principal_school_index = ["privatesch","schltype","stratio","schsize","totat","proatce","proat5ab"]
principal_school_index += ["proat5am","proat6","clsize","creactiv","edushort","staffshort","stubeha","teachbeha"]

In [10]:
# Grouping all questions into a giant list. 
all_question = home_index + self_index + parent_index + school_index + attitude_index + belonging_index
all_question += career_index + socio_status + principal_school_index

In [11]:
# Creating a dictionary of where key is student_id and value is the number of "No Response" the student provides.
"""
Explanation: After dropping all the unwanted columns, I will check which students to drop based on how many No 
Responses they provide. To speed up the computation, I transposed the dataframe so that each row refers to WLE
and each column refers to student_id. Afterwards, I used DataFrame's ability to filter responses based on values
inside each cell. Because there are around 30 WLEs, I chose to drop any student if the number of No Response WLE
is greater than the variable 'bar' (set to 3). 

The assumption behind this procedure is that students who provide a lot of No Response might not provide the most
reliable source of data. Furthermore, the number of students whose No Response exceed 'bar' are approximately 300
out of 8600 students in total. This supports the case that the benefit incurred from removing these 'No Response'
students from the analysis would outweigh the accuracy gained by having more data. 
"""

no_resp_Skey = {}
tempdf = thadf.T
tempdf = tempdf.loc[all_question]

for index in tempdf.columns.tolist():
    num_Sempty = tempdf[tempdf[index] == "No Response"].shape[0]
    no_resp_Skey[index] = num_Sempty

In [12]:
# Creating a list of students to drop (i.e. those whose number of No Response exceeds bar.)
drop_lst = []
bar = 3
for index in list(no_resp_Skey.keys()):
    if no_resp_Skey[index] >  bar:
        drop_lst.append(index)

In [13]:
# Dropping 'No Response' students. 
try: 
    thadf = thadf.drop(drop_lst)
except:
    print("No Response students already dropped")

In [14]:
# Imputing the data by filling in the average. 
"""
Explanation: At this stage, each of the 8300 students has at most three No Response. Removing every student with No 
Response would be an overkill given that there are approximately 2500 students with at least one No Response. I decided
to impute each No Response cell with the average of the column in which it resides for two reasons:

1.) Because I intend to use the multiple linear regression to analyze the data, imputing the No Response cell with 
each  column's average would not increase variance. After all, the regression model is based on reducing the sum of 
mean-squared difference between the average and the actual data. 

2.) Each student has at most three No Response, and those No Response cells spread across multiple columns. Therefore,
the imputation would not cause any column to significantly deviate from its original form. Running regression using
each column index as a predictor variable with/without imputation would likely yield similar statistical significance.

Note: Admittedly, I am uncertain about how to proceed with this process. Although there are some statistical reasons
(as provided above) why imputing with average will not change the result on each individual variable, I have 
reservations on whether applying this process on every variable altogether will affect the regression result. 

"""

for question in thadf.columns.tolist():
    average = avg(thadf[question])
    for index in thadf.index.tolist():
        if thadf.loc[index, question] == "No Response":
            thadf.loc[index, question] = average

In [15]:
# Creating a new dataframe because some of cells are strings of numbers rather than numbers themselves (i.e. '4.3'
# instead of 4.3). The new dataframe will be temporarily stored at newdf.

data = {}
for col_name in thadf.columns.tolist():
    temp_lst = thadf[col_name].tolist()
    new_lst = create_float_lst(temp_lst)
    data[col_name] = new_lst

newdf = pd.DataFrame(data)

In [17]:
# Dropping every column whose cells are 'Not applicable' and resetting thadf to the type-converted dataframe. 

drop_lst = ["bodyima", "paschpol", "soconpa", "presupp", "joyreadp", "cursupp", "emosupp", "pqschool"]
try:
    newdf = newdf.drop(drop_lst, axis = 1)
except:
    print("Not applicable axis already dropped")
    
thadf = newdf

In [18]:
# Creating a list of predictor variables to run the regression model.  
"""
Explanation: I dropped columns ["clsize"] from being run in the regression model because class size is already 
captured in the student to teacher ratio, so adding class size would create the multicollinearity problem.  
"""

regress_var_lst = []
for question in all_question:
    if question not in ["clsize"] and question in thadf.columns.tolist():
        regress_var_lst.append(question)  

In [19]:
# Creating a dataframe to run regression on. 
regress_df = pd.get_dummies(thadf[regress_var_lst + subject_lst])
regress_df

homepos  cultposs  hedres  wealth  ictres  joyread  screadcomp  \
0     -0.8374   -1.7576 -0.9779 -0.4710 -0.3490   0.5572   -0.825000   
1      2.7697    1.9522  1.1793  2.3512  1.2943  -0.0996   -0.105600   
2      0.9426   -0.2159 -0.1357  1.2248  0.3875   0.0127   -1.098500   
3     -0.6207   -0.2771 -1.0752 -0.4267 -0.3490  -0.2204   -1.098500   
4     -0.0068    0.1190 -1.4469  0.0721 -0.1536  -1.0727   -1.354300   
5     -0.2896    0.2690  0.0477 -0.5420 -0.1536   0.6934    0.122200   
6      1.0285    0.0082 -0.1357  1.0681  0.1350   0.3252    0.122200   
7      0.7794    0.2836  1.1793  1.0603  1.2943  -0.3709   -0.849000   
8      0.6161   -0.3471 -0.1357  0.9062  0.3875   0.8628    0.122200   
9      0.3174    1.9522  1.1793 -0.3104 -0.1536   0.9523    0.122200   
10     1.0771    1.9522  1.1793  0.3813  0.0595   1.6946    1.237400   
11    -0.3999   -0.4233  0.1806 -0.5258 -0.0440   0.1372   -1.443900   
12     0.2968    1.9522  1.1793 -0.3546 -0.3918   0.9908    0.122200   
13     1.9480    0.1190  1.1793  1.2248  0.6901   0.5915    1.883900   
14     0.9114    0.6860  1.1793  0.4937  0.6901   1.4712    0.663200   
15    -0.1662   -0.1916 -0.1357 -0.2340 -0.3490   0.9069   -0.219574   
16    -1.2598   -0.4738 -0.6947 -1.3991 -2.0184   1.4712    0.122200   
17     1.8407    1.9522 -0.1357  1.1958  0.3875   0.7546    1.065700   
18    -0.1931   -0.5721 -0.6947  0.0242 -0.3490   1.0123   -1.098500   
19    -1.4134    0.0644 -0.9779 -1.3950 -1.7005   0.9324    0.122200   
20     1.0267    0.6860  1.1793  0.4058  1.2943   1.2316    0.122200   
21    -0.8100   -0.2771  1.1793 -1.0499 -1.1460   0.6934    0.122200   
22     0.0294   -0.0667  1.1793 -0.2761 -0.3918   1.1891    0.122200   
23    -0.2438    0.0082 -0.1357 -0.5893 -1.4850   0.0995   -0.825000   
24    -0.7617   -0.0782 -1.3835 -0.4217 -0.7308  -0.4868   -1.180600   
25     0.3031   -0.2159  1.1793 -0.3203  0.4826  -1.0727   -1.470400   
26     0.2769   -0.4233  1.1793  0.3031 -0.1536  -0.0573   -0.219574   
27    -0.5202    0.0630  1.1793 -1.1793 -0.5986   1.7513    1.237400   
28     0.4700   -0.3751  1.1793  0.1975  0.0595   1.0781    0.663200   
29     0.4521    0.3619 -0.1357  0.3031 -0.3490   0.3568   -0.849000   
...       ...       ...     ...     ...     ...      ...         ...   
8277   0.6122    0.0630  0.0310  0.1377  0.9144   0.6380   -0.408900   
8278  -1.3261    0.2158  0.0310 -1.8189 -0.3918   1.3555    1.237400   
8279  -1.7946   -0.4233 -1.7257 -1.7759 -0.3490  -0.2008   -0.408900   
8280  -0.3602   -0.2144  1.1793 -0.4868 -0.3629   0.1372    0.122200   
8281  -1.1515   -0.4233  0.0477 -1.2223 -0.8694   0.5886   -0.825000   
8282  -1.1067   -0.3452 -1.0527 -0.9694 -1.3654   0.2363    0.122200   
8283  -1.6995   -0.4233 -1.3904 -1.5609 -1.5044   0.4397   -0.548500   
8284  -1.3261    0.3963  1.1793 -1.8524 -1.5913  -1.2321   -0.548500   
8285  -1.4195   -0.6712 -0.4903 -1.4242 -1.6927  -0.4556   -0.849000   
8286  -1.3452   -0.6712  0.0310 -1.5429 -0.9400   0.3568   -0.548500   
8287  -0.9882   -0.4738 -0.6853 -1.0409 -0.6068   0.8628    0.122200   
8288  -0.2601    0.3963 -0.5492 -0.2768 -0.6231   0.1539    0.699400   
8289  -1.4140   -0.0460 -0.6947 -1.6295 -1.7005   1.0123    0.699400   
8290  -0.0047   -0.1795 -0.1357  0.4443 -0.2993   0.7031   -0.105600   
8291  -0.2128   -0.3471 -0.1357 -0.1433 -0.1120   0.1372    1.883900   
8292  -0.7235   -0.2771 -0.3487 -0.7864 -0.1536   0.5836   -0.408900   
8293  -1.8729   -1.7576 -1.2862 -1.6192 -1.4787   0.1372    0.122200   
8294  -1.8280    0.2028 -1.9560 -1.5434 -1.2275   0.5230   -1.098500   
8295  -0.1284    0.3963 -1.9435  0.6628  0.7693   0.4871    1.883900   
8296  -0.1760    0.0082 -1.0752  0.1240 -0.3490   1.3052   -0.219574   
8297  -1.0874   -0.2159 -0.5492 -1.4788 -0.6231   0.3252    0.663200   
8298  -2.4633   -1.7576 -2.6530 -1.7563 -1.5550  -0.2704   -1.098500   
8299   1.5117    0.5060  1.1793  0.8857  0.3330   0.1372    0.122200   
8300   0.0730    0.1775 -0.8532  0.0695 -0.125

In [20]:
# Removing privatesch and schltype from the regress_var_lst. Also, each student's education should not be included
# as a predictor variable; hence, the condition member not in subject_lst
temp_lst = []
for member in regress_df.columns.tolist():
    if member != "privatesch" and member != "schltype" and member not in subject_lst:
        temp_lst.append(member)
regress_var_lst = temp_lst 

In [21]:
# Running the multiple linear regression model using math as a dependent variable to explore which variables
# we might need to filter out. 

X = regress_df[regress_var_lst]
y = regress_df["math"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   math   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.553
Method:                 Least Squares   F-statistic:                     210.8
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        09:01:28   Log-Likelihood:                -46141.
No. Observations:                8307   AIC:                         9.238e+04
Df Residuals:                    8257   BIC:                         9.273e+04
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                   263.3005      3.797     69.350      0.000     255.858     270.743
homepos                                  33.7221      4.343      7.765      0.000      25.209      42.235
cultposs                                 -7.5658      1.325     -5.712      0.000     -10.162      -4.969
hedres                                   -4.3432      1.237     -3.511      0.000      -6.768      -1.918
wealth                                  -31.8880      3.267     -9.761      0.000     -38.292     -25.484
ictres                                    8.5916      1.511      5.686      0.000       5.630      11.554
joyread                                   4.6690      1.234      3.784      0.000       2.250       7.088
screadcomp                               -0.0466      1.122     -0.042      0.967      -2.245       2.152
screaddiff                               -5.8267      1.005     -5.800      0.000      -7.796      -3.858
pisadiff                                 -2.7735      0.907     -3.056      0.002      -4.552      -0.995
emosups                                   2.2915      0.966      2.372      0.018       0.398       4.185
disclima                                  3.6059      0.946      3.812      0.000       1.751       5.460
teachsup                                  0.2831      1.031      0.274      0.784      -1.739       2.305
dirins                                   -6.0142      0.910     -6.607      0.000      -7.799      -4.230
stimread                                 -1.2933      0.970     -1.334      0.182      -3.194       0.607
adaptivity                                3.9600      0.929      4.262      0.000       2.139       5.781
teachint                                 -5.0914      1.117     -4.557      0.000      -7.281      -2.901
perfeed                                   0.4102      0.950      0.432      0.666      -1.451       2.272
percomp                                   4.0249      0.871      4.622      0.000       2.318       5.732
percoop                                   2.6361      0.882      2.987      0.003       0.906       4.366
attlnact                                 -0.6106      0.877     -0.696      0.486      -2.329       1.108
compete                                  -1.3523      0.989     -1.367      0.172      -3.292       0.587
workmast                                  9.5015      1.014      9.366      0.000       7.513      11.490
gfofail                                   6.7219      0.866      7.759      0.000       5.024       8.420
eudmo                                   -12.8372      1.078    -11.906      0.000     -14.951     -10.724
swbp                                      3.2456      0.796      4.076      0.000       1.685       4.806
resilienc

In [22]:
# Examples of how I chose which variables will be kept in the final regression model.
"""
The procedure I used is as follows:

1.) Run regression on every variable pertaining to a factor (e.g. home_index for family background). The result
below shows many counter-intuitive results, such as cultposs having negative coefficient. The more cultural
capital a student possesses, the better her education performance should be. 

My hypothesis: Some of these variables might strongly correlate with one another, causing the multicollinearity
problem (i.e. the interaction between two positively correlated variables allows the minimum R-squared to be 
minimized by adjusting both variables' coefficients into different directions).

2.) To test this hypothesis, I run regression on homepos and cultposs separately, each of which serves as 
the only dependent variable in the regression model. The result: both variables have positive coefficients.

3.) Afterwards, I experimented with running cultposs and homepos altogether. The coefficient for cultposs 
in the two-variable model becomes negative. This supports our hypothesis. I also used Pearson's coefficient
to check the correlation as well: variables with high correlation will nt

4.) To decide which variable to keep, I experimented with using each index as a predictor variable individually.
Whichever index yields the highest coefficient and/or highest R-squared value will be included in the final 
model. Moreover, if the sum of R-squared from regressing two (or more) variables pertaining to the same factor separately
(e.g. "Enjoyment in reading" and"Perception of difficulty in reading" for self-experience) is roughly equal
to running both altogether, then both variables will be included as well. 

Reason: The sum of individual R-squared approximating R-squared with many variables suggests those 
variables might not be strongly correlated.

5.) Repeat Process 1-4 for every factor. 
"""

'\nThe procedure I used is as follows:\n\n1.) Run regression on every variable pertaining to a factor (e.g. home_index for family background). The result\nbelow shows many counter-intuitive results, such as cultposs having negative coefficient. The more cultural\ncapital a student possesses, the better her education performance should be. \n\nMy hypothesis: Some of these variables might strongly correlate with one another, causing the multicollinearity\nproblem (i.e. the interaction between two positively correlated variables allows the minimum R-squared to be \nminimized by adjusting both variables\' coefficients into different directions).\n\n2.) To test this hypothesis, I run regression on homepos and cultposs separately, each of which serves as \nthe only dependent variable in the regression model. The result: both variables have positive coefficients.\n\n3.) Afterwards, I experimented with running cultposs and homepos altogether. The coefficient for cultposs \nin the two-variable 

In [23]:
# Example of Step 1: Running regressions on index pertaining to family background.
X = regress_df[home_index]
y = regress_df["math"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   math   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.269
Method:                 Least Squares   F-statistic:                     613.1
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        09:01:48   Log-Likelihood:                -48205.
No. Observations:                8307   AIC:                         9.642e+04
Df Residuals:                    8301   BIC:                         9.646e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        476.9125      1.291    369.500      0.000     474.382     479.443
homepos       88.6111      5.323     16.648      0.000      78.177      99.045
cultposs     -16.9278      1.668    -10.147      0.000     -20.198     -13.658
hedres        -2.9884      1.560     -1.915      0.056      -6.047       0.070
wealth       -56.4688      4.113    -13.730      0.000     -64.531     -48.407
ictres        12.3511      1.915      6.450      0.000       8.598      16.105
==============================================================================
Omnibus:                       38.544   Durbin-Watson:                   1.322
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               38.953
Skew:                           0.164   Prob(JB):                     3.48e-09
Kurtosis:                       2.932   Cond. No.                         23.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# Example of Step 2: Running regression on cultposs/homepos separately. Both variables yield positive coefficients.
X = regress_df[["homepos"]]
y = regress_df["math"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   math   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     2612.
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        09:01:51   Log-Likelihood:                -48375.
No. Observations:                8307   AIC:                         9.675e+04
Df Residuals:                    8305   BIC:                         9.677e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        480.0676      1.199    400.442      0.000     477.718     482.418
homepos       38.6645      0.756     51.111      0.000      37.182      40.147
==============================================================================
Omnibus:                       55.373   Durbin-Watson:                   1.281
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               55.833
Skew:                           0.193   Prob(JB):                     7.52e-13
Kurtosis:                       2.888   Cond. No.                         2.57
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
# Example of Step 3: Running cultposs and homepos altogether. Observe that cultposs coefficient changes from positive
# negative. 

X = regress_df[["cultposs", "homepos"]]
y = regress_df["math"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   math   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     1315.
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        09:01:53   Log-Likelihood:                -48368.
No. Observations:                8307   AIC:                         9.674e+04
Df Residuals:                    8304   BIC:                         9.676e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        480.6434      1.207    398.073      0.000     478.277     483.010
cultposs      -4.8882      1.287     -3.799      0.000      -7.411      -2.366
homepos       40.5372      0.902     44.920      0.000      38.768      42.306
==============================================================================
Omnibus:                       55.647   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.109
Skew:                           0.193   Prob(JB):                     6.55e-13
Kurtosis:                       2.887   Cond. No.                         3.11
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Example of Step 4: Because adding cultposs does not improve R-squared values from running regression merely on 
# homepos, I chose to only include homepos. Note that in the final result, I chose to include only escs (Socioeconomic
# background). 
keep_lst = ["homepos"]

In [27]:
# Final result: Only keep variables that have high-coefficients and yield high R-squared individually. 

keep_lst = ["joyread","screaddiff"]
keep_attitude_index = ["workmast","gfofail"] 
keep_belonging_index = ["belong","discrim"] # Pick discrim
keep_escs = ["escs"]
keep_principal_index = ["creactiv","edushort","stratio","stubeha"] # creactiv highly correlated with percoop
keep_school_type = ["privatesch_private"]
keep_lst += keep_attitude_index + keep_belonging_index + keep_escs + keep_principal_index + keep_school_type

X = regress_df[keep_lst]
y = regress_df["read"]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X) 

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   read   R-squared:                       0.514
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     731.9
Date:                Thu, 23 Jan 2020   Prob (F-statistic):               0.00
Time:                        09:01:59   Log-Likelihood:                -46076.
No. Observations:                8307   AIC:                         9.218e+04
Df Residuals:                    8294   BIC:                         9.227e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                422.5999      3.722    113.547      0.000     415.304     429.896
joyread               14.9029      1.086     13.720      0.000      12.774      17.032
screaddiff            -9.8139      0.915    -10.721      0.000     -11.608      -8.019
workmast              13.5416      0.836     16.193      0.000      11.902      15.181
gfofail               11.6909      0.833     14.030      0.000      10.057      13.324
belong                 6.8713      1.109      6.197      0.000       4.698       9.045
discrim              -20.3520      0.720    -28.284      0.000     -21.763     -18.942
escs                  20.8238      0.607     34.311      0.000      19.634      22.013
creactiv              11.9978      0.943     12.718      0.000      10.149      13.847
edushort              -8.9492      0.708    -12.646      0.000     -10.336      -7.562
stratio               -0.6709      0.100     -6.709      0.000      -0.867      -0.475
stubeha              -14.1096      0.736    -19.179      0.000     -15.552     -12.668
privatesch_private   -16.0957      2.157     -7.462      0.000     -20.324     -11.867
==============================================================================
Omnibus:                        4.826   Durbin-Watson:                   1.517
Prob(Omnibus):                  0.090   Jarque-Bera (JB):                4.481
Skew:                           0.014   Prob(JB):                        0.106
Kurtosis:                       2.890   Cond. No.                         113.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# Check the Pearson's correlation between variables to help decide which variables are linearly correlated.    

data1 = thadf["workmast"].tolist()
data2 = thadf["gfofail"].tolist()


# calculate Pearson's correlation
corr, _ = pearsonr(data1, data2)
print('Pearson\'s correlation: %.3f' % corr)

Pearson's correlation: 0.148


In [29]:
# Saving the cleaned and inputed data to .csv 
thadf.to_csv("pisa2018_studentindex_perf_clean.csv")